In [1]:
#import base libraries
import pandas as pd
import numpy as np
import datetime
import copy

#misc
import warnings
warnings.filterwarnings('ignore')

# Data Cleaning Notebook

In [ ]:
# Import Dataset - Monday
data = pd.read_excel('OceanWise Info/BCCSN_Sightings_Master.xlsx', parse_dates= ['Time'])

#Check shape
print(data.shape)

#Print head
data.head()

In [3]:
#Check percentage of missing values in target
data[['Location Accuracy']].isnull().sum()/len(data)
#data[['Location Accuracy']].notnull().sum()/len(data)

Location Accuracy    0.366505
dtype: float64

As this project is about Proof of Concept, I am dropping the rows were Location Accuracy is null.

As this feature is my target and 36.6% is missing, I do not want to take the risk of going in and addressing these issues as it could skew the data.

In [5]:
#Find rows that have no location description and are Within 2 km Accuracy
missing_targetvalues = data.loc[data['Location Accuracy'].isnull()]

#print shape
print(missing_targetvalues.shape)

#Save missing_targetvalues as different csv
missing_targetvalues.to_csv('CSVs/Location_Accuracy/missing_targetvalues.csv')

(7620, 60)


Can use these in future to predict on.

In [ ]:
#create deepcopy and drop rows where Location Accuracy was Null
df = copy.deepcopy(data.dropna(subset=['Location Accuracy']))

#print shape
print(df.shape)

#check column names
df.columns

In [8]:
#Drop unnesscessary columns and columns missing too much information
df = df.drop(['SubDate', 'SubTime', 'ReportId', 'SourceName', 'Ecotype', 'Number of Animals Max', 
            'VesselName', 'Comments', 'OrganizationName','WhenInWater', 'HowYouHeard', 
            'HowYouHeardOtherDescription'], axis = 1)

In [9]:
#Check for nulls on reduced dataframe
df.isnull().sum()/len(df) * 100

SpeciesName              0.000000
SpeciesCategory          0.000000
LatitudeDD               0.000000
LongitudeDD              0.000000
Date                     0.000000
Time                     0.007592
IdConfidence             0.000000
LocationDescription      0.022777
SeaState                 0.000000
WindSpeed                0.000000
AnimalCountMeasure       0.000000
NumberOfAnimals          0.000000
Number of Animals Min    1.366639
Behaviour                0.007592
Source                   0.083517
SightingDistance         0.015185
Direction of Travel      0.007592
SightingPlatform         0.007592
DistressedState          0.000000
Experience               0.000000
WhyInWater               0.000000
Location Accuracy        0.000000
Year                     0.000000
Month                    0.000000
dtype: float64

In [10]:
#Check dtypes
df.dtypes

SpeciesName                      object
SpeciesCategory                  object
LatitudeDD                      float64
LongitudeDD                     float64
Date                     datetime64[ns]
Time                             object
IdConfidence                     object
LocationDescription              object
SeaState                         object
WindSpeed                        object
AnimalCountMeasure               object
NumberOfAnimals                  object
Number of Animals Min            object
Behaviour                        object
Source                           object
SightingDistance                 object
Direction of Travel              object
SightingPlatform                 object
DistressedState                  object
Experience                       object
WhyInWater                       object
Location Accuracy                object
Year                              int64
Month                             int64
dtype: object

## Letter Casing and Punctuation Fixing

All the below categories can be easily view on the Whale Sightings App: Whale Report

In [11]:
#Fixing SeaState
df['SeaState'] = df['SeaState'].str.replace('light', 'Light', regex = True)
df['SeaState'] = df['SeaState'].str.replace(',Wind', 'Wind', regex = True)
df['SeaState'] = df['SeaState'].str.replace('Light air,  small ripples', 'Light air, small ripples', regex = True)

In [12]:
#Check value counts
df.SeaState.value_counts()

Light air, small ripples                                          4406
Wind felt on face;wave crests have glassy appearance              3139
Sea is smooth                                                     2183
Crests begin to break;whitecaps start to form at 10 knots         1744
-                                                                  914
Frequent whitecaps - seas to 3 ft (1m)                             533
Moderate waves form 4-5 ft (1.5-2.5 m);some spray carried          160
Exceptionally high waves to 30 ft (9m)                              46
Large waves 5-7 ft (2-3m);more spray                                28
Swell forms 8-10 ft (3-4m);foam blown in streaks                    11
Moderately high waves to 16 ft(5m);crests break into spindrift       4
High waves to 20 ft (7m);dense foam, visibility reduced              2
Waves to 35 ft (11m);very limited visibility                         1
Name: SeaState, dtype: int64

In [13]:
#Fixing WindSpeed
df['WindSpeed'] = df['WindSpeed'].str.replace('5 - 6 kts', '4 - 6 kts', regex = True)
df['WindSpeed'] = df['WindSpeed'].str.replace('2 - 3 kts', '1 - 3 kts', regex = True)
df['WindSpeed'] = df['WindSpeed'].str.replace('3 - 3 kts', '1 - 3 kts', regex = True)
df['WindSpeed'] = df['WindSpeed'].str.replace('18 - 21 kts', '17 - 21 kts', regex = True)

In [14]:
#Check value counts
df.WindSpeed.value_counts()

1 - 3 kts (Light Air)            3092
4 - 6 kts (Light Breeze)         2729
7 - 10 kts (Gentle Breeze)       2504
11 - 16 kts (Moderate Breeze)    1597
-                                1469
0 kts (Calm)                     1106
17 - 21 kts (Fresh Breeze)        466
22 - 27 kts (Strong Breeze)       167
28 - 33 kts (Near Gale)            29
34 - 40 kts (Gale)                  9
41 - 47 kts (Strong Gale)           2
48 - 55 kts (Storm)                 1
Name: WindSpeed, dtype: int64

In [15]:
#Fixing Direction of Travel
df['Direction of Travel'] = df['Direction of Travel'].str.replace('Uknown', 'Unknown', regex = True)
df['Direction of Travel'] = df['Direction of Travel'].str.replace('Unknown Travel Direction', 'Unknown', regex = True)

In [16]:
#Check value counts
df['Direction of Travel'].value_counts()

Various       3230
North         1506
South         1468
East          1336
West          1255
North-West    1093
South-East     827
Unknown        701
North-East     649
South-West     564
-              484
___             57
Name: Direction of Travel, dtype: int64

In [17]:
#Fixing Sighting Platofrm
df['SightingPlatform'] = df['SightingPlatform'].str.replace('vessel', 'Vessel', regex = True)
df['SightingPlatform'] = df['SightingPlatform'].str.replace('Verry', 'Ferry', regex = True)
df['SightingPlatform'] = df['SightingPlatform'].str.replace('feet', 'ft', regex = True)
df['SightingPlatform'] = df['SightingPlatform'].str.replace('> 25 ft', '25 - 60 ft', regex = True)
df['SightingPlatform'] = df['SightingPlatform'].str.replace('25-60', '25 - 60', regex = True)
df['SightingPlatform'] = df['SightingPlatform'].str.replace('Lightstation', 'Land (close to water)', regex = True)

In [18]:
#Check value counts
df.SightingPlatform.value_counts()

Motor Vessel 25 - 60 ft    3645
Land (close to water)      2456
Ferry                      2119
Motor Vessel > 60 ft       1456
Motor Vessel < 25 ft       1225
Sailboat                    820
Land (high above water)     692
Other                       406
-                           273
Kayak                        57
Airplane / Helicopter        19
___                           2
Name: SightingPlatform, dtype: int64

In [19]:
#Fix DistressedState
df['DistressedState'] = df['DistressedState'].str.replace('No ', 'No', regex = True)

In [20]:
#Check value counts
df.DistressedState.value_counts()

No     13148
Yes       23
Name: DistressedState, dtype: int64

In [21]:
#Fix Experience
df['Experience'] = df['Experience'].str.replace('Recreational', 'Novice recreational observer', regex = True)

In [22]:
#Check value counts
df.Experience.value_counts()

Ecotourism professional              5186
Other marine professional            2941
Novice recreational observer         1987
Experienced recreational observer    1783
Researcher                            876
-                                     378
Lightstation keeper                    17
___                                     3
Name: Experience, dtype: int64

In [23]:
#Fix Location Accuracy (Target)
df['Location Accuracy'] = df['Location Accuracy'].str.replace('w', 'W', regex = True)
df['Location Accuracy'] = df['Location Accuracy'].str.replace('WI', 'Wi', regex = True)
df['Location Accuracy'] = df['Location Accuracy'].str.replace('m ', 'm', regex = True)

## Null values
Most columns are categorical and isnull() is not picking up on empty strings. On closer expection, empty strings are '-' or '___'. So I will first handle these

In [25]:
# Example of how empty strings were found in Locations description
print('No. of missing values in Location Description:', (df['LocationDescription'].values == '-').sum())

#Checking missing values in other category columns
print('No. of missing data in SpeciesName:', (df['SpeciesName'].values == '-').sum())
print('No. of missing data in SpeciesCategory:', (df['SpeciesCategory'].values == '-').sum())
print('No. of missing data in IdConfidence:', (df['IdConfidence'].values == '-').sum())
print('No. of missing data in SeaState:', (df['SeaState'].values == '-').sum())
print('No. of missing data in WindSpeed:', (df['WindSpeed'].values == '-').sum())
print('No. of missing data in Behaviour:', (df['Behaviour'].values == '-').sum())
print('No. of missing data in Source:', (df['Source'].values == '-').sum())
print('No. of missing data in SightDistance:', (df['SightingDistance'].values == '-').sum())
print('No. of missing data in Direction of Travel:', (df['Direction of Travel'].values == '-').sum())
print('No. of missing data in SightingPlatform:', (df['SightingPlatform'].values == '-').sum())
print('No. of missing data in DistressedState:', (df['DistressedState'].values == '-').sum())
print('No. of missing data in Experience:', (df['Experience'].values == '-').sum())
print('No. of missing data in WhyInWater:', (df['WhyInWater'].values == '-').sum())
print('No. of missing data in Location Accuracy:', (df['Location Accuracy'].values == '-').sum())

No. of missing values in Location Description: 8028
No. of missing data in SpeciesName: 0
No. of missing data in SpeciesCategory: 0
No. of missing data in IdConfidence: 151
No. of missing data in SeaState: 914
No. of missing data in WindSpeed: 1469
No. of missing data in Behaviour: 1016
No. of missing data in Source: 0
No. of missing data in SightDistance: 5905
No. of missing data in Direction of Travel: 484
No. of missing data in SightingPlatform: 273
No. of missing data in DistressedState: 0
No. of missing data in Experience: 378
No. of missing data in WhyInWater: 1014
No. of missing data in Location Accuracy: 0


In [27]:
#Replace blank strings with NaN
missing_values = ['-', '___']
df = df.replace(missing_values, np.NaN)

In [28]:
#Check percentage of null again
df.isnull().sum()/len(df) * 100

SpeciesName               0.000000
SpeciesCategory           0.098702
LatitudeDD                0.000000
LongitudeDD               0.000000
Date                      0.000000
Time                      0.007592
IdConfidence              1.154051
LocationDescription      61.012831
SeaState                  6.939488
WindSpeed                11.153291
AnimalCountMeasure        0.000000
NumberOfAnimals           1.351454
Number of Animals Min     3.355858
Behaviour                 8.625009
Source                    0.083517
SightingDistance         45.752031
Direction of Travel       4.115101
SightingPlatform          2.095513
DistressedState           0.000000
Experience                2.892719
WhyInWater                8.146686
Location Accuracy         0.000000
Year                      0.000000
Month                     0.000000
dtype: float64

In [30]:
#Applied another limitation on data based on BCCSN protocols to reduce potential bias in model
#Code missing here for privacy reasons

#First dropping Location Description and SightingDistance as too many missing values
df = df.drop(['LocationDescription', 'SightingDistance'], axis = 1)

#print new shape
df.shape

(6171, 22)

In [32]:
#Define list for category columns to replace values with most frequent
cat_col = ['SpeciesName', 'Time', 'IdConfidence', 'SeaState', 'WindSpeed',
            'AnimalCountMeasure', 'NumberOfAnimals', 'Behaviour', 'Source', 
            'Direction of Travel', 'SightingPlatform', 'DistressedState', 'Experience']
            
#Replace NaN with most frequent value for Category columns
for col in cat_col:
    most_freq = df[col].mode().values[0]
    df[col].fillna(most_freq, inplace=True)

In [33]:
#calculate mean and mode of number of animals min
print("Mean:", df['Number of Animals Min'].mean())
print("Mode:", df['Number of Animals Min'].mode())
print("Max:", df['Number of Animals Min'].max())

Mean: 4.241913746630728
Mode: 0    1.0
dtype: float64
Max: 250.0


I will go with the mean for the replacement value

In [34]:
#Replace missing values in Number of Animals Min
df['Number of Animals Min'].fillna(value=df['Number of Animals Min'].mean(), inplace = True)

### Applying condition on columns to replace nulls

Based on Experience, condition for WhyInWater is either Professional or Recreational

Based on SpeciesName, make new column for Species Category to replace nulls

In [35]:
#Fix WhyInWater column based on Experience
df['WhyInWater'] = np.where(df['Experience'].str.contains('recreational'), 'Recreational', 'Professional')

In [36]:
#Check WhyInWater Values
df.WhyInWater.value_counts()

Professional    3677
Recreational    2494
Name: WhyInWater, dtype: int64

In [37]:
#create list for species names under dolphins and whales

whales = ['Killer whale',"Grey whale", 'Humpback whale', 'Minke whale', 'Fin whale',  'Sperm whale', 
            'False killer whale', "Baird's beaked whale", "Cuvier's beaked whale", 'Sei whale', 
            'Blue whale', 'North Pacific right whale', 'Unidentified whale','Other rare species']

d_p = ['Harbour porpoise',"Dall's porpoise", 'Pacific white-sided dolphin', "Risso's dolphin",
        'Northern right whale dolphin','Unidentified dolphin or porpoise']

#Create empty list
speciescategory = []

#create for loop and appen to list
for n in df.SpeciesName:
    if (n in whales) == True:
        speciescategory.append('Whales')
    elif (n in d_p) == True:
        speciescategory.append('Dolphins and Porpoises')
    else:
        speciescategory.append('Sea Turtle')
        
#define new column
df['SpeciesCategory'] = speciescategory

In [38]:
df.SpeciesCategory.value_counts()

Whales                    5289
Dolphins and Porpoises     882
Name: SpeciesCategory, dtype: int64

In [39]:
#Check for anymore missing values
df.isnull().sum()

SpeciesName              0
SpeciesCategory          0
LatitudeDD               0
LongitudeDD              0
Date                     0
Time                     0
IdConfidence             0
SeaState                 0
WindSpeed                0
AnimalCountMeasure       0
NumberOfAnimals          0
Number of Animals Min    0
Behaviour                0
Source                   0
Direction of Travel      0
SightingPlatform         0
DistressedState          0
Experience               0
WhyInWater               0
Location Accuracy        0
Year                     0
Month                    0
dtype: int64

### Fix NumberOfAnimals

Split into 2 columns

In [ ]:
#View Number of Animals
df.NumberOfAnimals.unique()

In [41]:
#Create Empty Lists
minanimals = []
maxanimals = []

#For loop to check type
for x in df.NumberOfAnimals:
    #If integer append 0 in minanimal list
    if type(x) == int:
        minanimals.append(0) 
        maxanimals.append(x)
    elif type(x) == float:
        minanimals.append(0)
        maxanimals.append(x)
    #If str, split and append to minanimal & maxanimal
    else:
        new = x.split("~")
        minanimals.append(new[0]) 
        maxanimals.append(new[1]) 

In [42]:
#Add new columns with variables
df['MinAnimal'] = minanimals
df['MaxAnimal'] = maxanimals

#Convert to type int
df.MinAnimal = df.MinAnimal.astype(int)
df.MaxAnimal = df.MaxAnimal.astype(int)

In [43]:
print(f"Mean Min Animals: {df['MinAnimal'].mean()}")
print(f"Mode Min Animals: {df['MinAnimal'].mode()}")

print(f"Mean Max Animals: {df['MaxAnimal'].mean()}")
print(f"Mode Max Animals: {df['MaxAnimal'].mode()}")

Mean Min Animals: 0.5929346945389726
Mode Min Animals: 0    0
dtype: int64
Mean Max Animals: 4.681251012801815
Mode Max Animals: 0    1
dtype: int64


In [ ]:
#check to see if it worked
df.MinAnimal.unique()

In [ ]:
#View new columns
df[['MinAnimal','MaxAnimal']]

In [46]:
#Drop NumberOfAnimals
df = df.drop(['NumberOfAnimals'], axis = 1)

## Extracting features from Behaviour List Column

I want to be able to take these strings and create new columns based on if the animal is doing the behaviour or not. Meaning I am doing my own one-hot encode

In [47]:
#Fix Behaviour
df.Behaviour = df.Behaviour.str.replace(' Fluking', 'Fluking', regex = True)
df.Behaviour = df.Behaviour.str.replace('slow moving', 'Slow moving', regex = True)
df.Behaviour = df.Behaviour.str.replace('Slow moving', 'Slow moving', regex = True)

#extract Behaviour column from dataframe
behave = df.Behaviour.str.split(',')

#Extract all values from column and set the list
flat_list = [item for sublist in behave for item in sublist]
flat_list = list(set(flat_list))
print(flat_list)

['Feeding', 'Fast moving', 'Spy-hopping', 'Slow moving', 'Breaching', 'Bow-riding', 'Porpoising', 'Fluking', 'Depredation']


In [48]:
#turn behaviour column into list and split based on comma
ls = list(df['Behaviour'].str.split(','))

#create for loop do go through flat list (all unique values)
for element in flat_list:
    #create column for each element
    df[element] = 0
    #enumerate through the column turned ls
    for index, l in enumerate(ls):
        #Assign False as 0 and True as 1 if element in ls
        df[element].iloc[index] = int(element in l)

In [ ]:
#check to see if it worked
df[['Fast moving', 'Porpoising', 'Spy-hopping', 'Fluking', 'Slow moving', 
    'Feeding', 'Depredation', 'Bow-riding', 'Breaching']].head()

In [51]:
#Drop behaviour column
df = df.drop(['Behaviour'], axis = 1)

## Date, Time and Season Features

1. Turn date and time into datetime (SubTime already datetime)
2. Extract month, day, week, year from Data
3. Make a column for Seasons

In [57]:
#Create Hour Column
item = df.Time.apply(lambda x: str(x)[:2])
df['Hour'] = item

In [62]:
#Assign time of day to Hour

times = [(df['Hour'] == '00') | (df['Hour'] == '01') | (df['Hour'] == '02') | (df['Hour'] == '03') | (df['Hour'] == '04') | (df['Hour'] == '05'),
            (df['Hour'] == '06') | (df['Hour'] == '07') | (df['Hour'] == '08') | (df['Hour'] == '09') | (df['Hour'] == '10') | (df['Hour'] == '11'),
            (df['Hour'] == '12') | (df['Hour'] == '13') | (df['Hour'] == '14') | (df['Hour'] == '15') | (df['Hour'] == '16') | (df['Hour'] == '17'),
            (df['Hour'] == '18') | (df['Hour'] == '19') | (df['Hour'] == '20') | (df['Hour'] == '21') | (df['Hour'] == '22') | (df['Hour'] == '23')]
df['TimeOfDay'] = np.select(times, ['Early Morning', 'Morning', 'Afternoon', 'Night'])

In [63]:
#Check time of day
df.TimeOfDay.unique()

array(['Morning', 'Night', 'Afternoon', 'Early Morning'], dtype=object)

In [64]:
#Assign time of day to Hour with shorter spans

times2 = [(df['Hour'] == '00') | (df['Hour'] == '01') | (df['Hour'] == '02'),
         (df['Hour'] == '03') | (df['Hour'] == '04') | (df['Hour'] == '05'),
        (df['Hour'] == '06') | (df['Hour'] == '07') | (df['Hour'] == '08'),
        (df['Hour'] == '09') | (df['Hour'] == '10') | (df['Hour'] == '11'),
        (df['Hour'] == '12') | (df['Hour'] == '13') | (df['Hour'] == '14'),
        (df['Hour'] == '15') | (df['Hour'] == '16') | (df['Hour'] == '17'),
        (df['Hour'] == '18') | (df['Hour'] == '19') | (df['Hour'] == '20'),
        (df['Hour'] == '21') | (df['Hour'] == '22') | (df['Hour'] == '23')]
ls_times2 =  ['0-3', '3-6', '6-9', '9-12', '12-15', '15-18', '18-21', '21-24']
df['TimeOfDay_2'] = np.select(times2, ls_times2)

In [65]:
#Check Time of Day 2
df.TimeOfDay_2.unique()

array(['6-9', '18-21', '12-15', '9-12', '15-18', '21-24', '3-6', '0-3'],
      dtype=object)

In [53]:
#Create columns for Day, Day of Week (Shortened to 3 letters),Month & Year
df['Day'] = df['Date'].dt.day
df['WeekDay'] = df['Date'].dt.weekday #Monday = 0
# df['WeekDay'] = df['Date'].dt.day_name().str[:3]
df['Month'] = df['Date'].dt.month
#df['Month'] = df['Date'].dt.month_name().str[:3]
df['Year'] = df['Date'].dt.year
df['YearStr'] = df.Year.astype(str)

In [54]:
#create conditions for Seasons
conditions = [
            (df['Month'] <= 3), #winter
            (df['Month'] >= 4) & (df['Month'] <= 6), #spring
            (df['Month'] >= 7) & (df['Month'] <= 9), #summer
            (df['Month'] >= 10) & (df['Month'] <= 12)  #autumn
]

#name seasons
seasons = ['Winter', 'Spring', 'Summer', 'Autumn']

#Make season column
df['Season'] = np.select(conditions, seasons)

In [56]:
#Check to see if it worked
df.Season.unique()

array(['Summer', 'Spring', 'Autumn', 'Winter'], dtype=object)

## Feature Seletion

1. Reducing Categories in WindSpeed
2. Reducing Categories in SeaState

In [66]:
light =['1 - 3 kts (Light Air)', '4 - 6 kts (Light Breeze)', '7 - 10 kts (Gentle Breeze)']
breeze = ['11 - 16 kts (Moderate Breeze)', '17 - 21 kts (Fresh Breeze)', '22 - 27 kts (Strong Breeze)']
gale = ['28 - 33 kts (Near Gale)', '34 - 40 kts (Gale)', '41 - 47 kts (Strong Gale)']
storm = ['48 - 55 kts  (Storm)', '56 - 63 kts (Violent Storm)', '64+ kts (Hurricane)']

#Create empty list
windcategory = []

#create for loop and appen to list
for n in df.WindSpeed:
    if (n in light) == True:
        windcategory.append('Light')
    elif (n in breeze) == True:
        windcategory.append('Breeze')
    elif (n in gale) == True:
        windcategory.append('Gale')
    elif (n in storm) == True:
        windcategory.append('Storm')
    else:
        windcategory.append('Calm')
        
#define new column
df['WindCategory'] = windcategory

In [67]:
#Check new value counts
df.WindCategory.value_counts()

Light     4691
Breeze     927
Calm       527
Gale        26
Name: WindCategory, dtype: int64

In [68]:
crests = ['Wind felt on face;wave crests have glassy appearance', 'Crests begin to break;whitecaps start to form at 10 knots']
moderate = ['Frequent whitecaps - sea to 3 ft (1 m)', 'Moderate waves form 4-5 ft (1.5-2.5 m);some spray carried']
rough = ['Large waves 5-7 ft (2-3 m);more spray', 'Swell forms 8-10 ft (3-4 m);foam blown in streaks']
dangerous = ['Moderately high waves to 16 ft (5 m);crests break into spindrift',
            'High waves to 20 ft (7 m);dense foam, visibility reduced']
storm = ['Exceptionally high waves to 30 ft (9 m)','Waves to 35 ft (11 m);very limited visibility', 
        'Waves to 50 ft (15m);air filled with foam and spray']

#Create empty list
seacategory = []

#create for loop and append to list
for n in df.SeaState:
    if (n in crests) == True:
        seacategory.append('Crest')
    elif (n in moderate) == True:
        seacategory.append('Moderate')
    elif (n in rough) == True:
        seacategory.append('Rough')
    elif (n in dangerous) == True:
        seacategory.append('Dangerous')
    elif (n in storm) == True:
        seacategory.append('Storm')
    else:
        seacategory.append('Calm')
        
#define new column
df['SeaCategory'] = seacategory

In [69]:
#Check new value counts
df.SeaCategory.value_counts()

Calm        4077
Crest       2022
Moderate      72
Name: SeaCategory, dtype: int64

In [ ]:
#View New Dataframe
df.head()

## Check Target variable

Dataset is extremely unbalanced, majority of classes are Within 2km

In [ ]:
df['Location Accuracy'].value_counts()

In [72]:
#Create definition to group target variables
def location_accuracy(df):
    """
    Input - dataframe
    Output - Newly Assigned Target Values for Location Accuracy
    Function groups Location Accuracy values into 3 subcategories
    """

    #cat_1 = list of location accuracies < 500 m
    #cat_2 = list of location accuracies 500 m - 2km
    #cat_3 = list of location accuracies > 2 km

    for x in cat_1:
        df.loc[df['Location Accuracy'].str.contains(x, na=False), 'Location Accuracy'] = '< 500m'
    for x in cat_2:
        df.loc[df['Location Accuracy'].str.contains(x, na=False), 'Location Accuracy'] = '500m-2km'
    for x in cat_3:
        df.loc[df['Location Accuracy'].str.contains(x, na=False), 'Location Accuracy'] = '> 2km'

    return df

In [73]:
#Apply location_accuracy function
df = location_accuracy(df)

In [74]:
#Check new values
df['Location Accuracy'].value_counts()

< 500m      2748
500m-2km    2558
> 2km        865
Name: Location Accuracy, dtype: int64

# Change Dataframe into Ordinals for EDA

In [75]:
#Save final csv
df.to_csv('CSVs/Location_Accuracy/locationaccuracy_eda.csv', index = False)

In [ ]:
#Double check columns and change categoricals into ordinals
df.columns

In [79]:
#create ordinal plot so I can do Statistics
ordinal_df = df.replace({"SpeciesCategory": {"Whales":0, "Dolphins and Porpoises":1},
                    "IdConfidence": {"Certain": 0, "Probable": 1, 'Possible':2, 'Uncertain':3},
                    "Source": {"MOBILE":0, "WEB":1},
                    "DistressedState": {"No": 0, "Yes": 1},
                    "AnimalCountMeasure": {"Approximate": 0, "Exact": 1, "Range" : 2},
                    "Direction of Travel": {"West": 0, "North-West": 1, "North": 2, "North-East": 3, "East": 4,
                                        "South-East": 5, "South": 6, "South-West": 7, "Various": 8, "Unknown":9},
                    "SightingPlatform": {"Kayak":0, "Ferry":1, "Sailboat":2, "Motor Vessel < 25 ft": 3, 
                                        "Motor Vessel 25 - 60 ft": 4, "Motor Vessel > 60 ft": 5, 
                                        "Land (close to water)":6, "Land (high above water)":7,
                                        "Airplane / Helicopter":8, "Other":9},
                    "Experience": {"Researcher": 0, "Ecotourism professional": 1, "Other marine professional": 2,
                                   "Novice recreational observer": 3, "Experienced recreational observer": 4,
                                   "Lightstation keeper":5},
                    "WhyInWater" : {"Recreational":0, "Professional": 1},
                    "Season" : {'Winter':0, 'Spring': 1, "Summer": 2, "Autumn":3, "Winter":4},
                    "WindCategory": {"Calm":0, "Light":1, "Breeze":2, "Gale":3},
                    "SeaCategory": {"Calm":0, "Crest":1, "Moderate":2},
                    "TimeOfDay" : {'Early Morning': 0, "Morning": 1, "Afternoon": 2, "Night": 3},
                    "TimeOfDay_2": {'0-3':0, '3-6':1, '6-9':2, '9-12':3, '12-15':4, 
                                    '15-18':5, '18-21':6, '21-24':7},
                    "Location Accuracy": {'< 500m':0, '500m-2km':1, '> 2km':2}
                    
})

In [ ]:
#check final ordinal dataframe
ordinal_df.head()

In [81]:
#Save Ordinal Dataframe as CSV
ordinal_df.to_csv('CSVs/Location_Accuracy/ordinal_locationaccuracy.csv', index = False)